# PLCTestbench

In [1]:
from plctestbench.settings import *
from plctestbench.plc_testbench import PLCTestbench
from plctestbench.loss_simulator import BinomialPLS, GilbertElliotPLS
from plctestbench.plc_algorithm import AdvancedPLC, ZerosPLC, LastPacketPLC, LowCostPLC, BurgPLC, ExternalPLC
from plctestbench.output_analyser import MSECalculator, MAECalculator, PEAQCalculator

from plctestbench.worker import OriginalAudio

testbench_settings = {
    'root_folder': 'C:\\Data\\personale\\Università\\2022-2023\\original_tracks',
    'db_ip': 'localhost',
    'db_username': 'root',
    'db_password': 'Marmolada3343',
}

original_audio_tracks = [(OriginalAudio, OriginalAudioSettings('Blues_Bass.wav')),
                         (OriginalAudio, OriginalAudioSettings('Blues_Guitar.wav'))]

packet_loss_simulators = [(GilbertElliotPLS, GilbertElliotPLSSettings()),
                          (BinomialPLS, BinomialPLSSettings())]

crossfade_settings = [QuadraticCrossfadeSettings(length=50),
                               QuadraticCrossfadeSettings(length=5),
                               QuadraticCrossfadeSettings(length=5)]

crossfade_frequencies = [200, 2000]

band_settings = {'mid':
                 [BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=50)], order=1024),
                 BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=5)], order=512),
                 BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=1)], order=256)],
                 'side':
                 [BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=50)], order=1024),
                 BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=1)], order=256)]}

plc_algorithms = [(AdvancedPLC, AdvancedPLCSettings(band_settings, frequencies = {'mid': [200, 2000], 'side': [1000]}, channel_link=False, stereo_image_processing = StereoImageType.mid_side)),
                  (ZerosPLC, ZerosPLCSettings(crossfade_frequencies=crossfade_frequencies, fade_in=[QuadraticCrossfadeSettings(length=10)], crossfade=crossfade_settings)),
                  (LastPacketPLC, LastPacketPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=10)], mirror_x=True, mirror_y=True, clip_strategy=ClipStrategy.clip)),
                  (LowCostPLC, LowCostPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=10)])),
                  (BurgPLC, BurgPLCSettings(fade_in=[QuadraticCrossfadeSettings(length=1)], crossfade=[QuadraticCrossfadeSettings(length=10)]))]

metrics = [(MSECalculator, MSECalculatorSettings()),
           (MAECalculator, MAECalculatorSettings())
           #,(PEAQCalculator, PEAQCalculatorSettings())
           ]

testbench = PLCTestbench(original_audio_tracks, packet_loss_simulators, plc_algorithms, metrics, testbench_settings)

User already exists in the database.


KeyError: 'The key order is already present in the settings.'

In [ ]:
testbench.run()

In [ ]:
%matplotlib inline
testbench.plot(to_file=True, original_tracks=True, lost_samples_masks=True, output_analyses=True)